In [57]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [58]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import os
import glob
import datetime
import pytz
import seaborn as sns
from src.apis import zeroshot_api
import pprint
import datetime
import time
import pytz

warnings.filterwarnings('ignore')

In [59]:
path =  "./data"

###choose GTT model variant
modelchoice = 'large'
modelpath = f'./GTT/checkpoints/GTT-{modelchoice}'
baseContextLength = 288

###whether do 1st-order differencing before prediction, sometimes set autodiff=True will improve prediction accuracy
autodiff = True
PLOT = False
splits = [('2016-02-01', '2016-03-01'), ('2016-03-01', '2016-04-01'), ('2016-04-01', '2016-05-01'),
          ('2016-05-01', '2016-06-01'), ('2016-06-01', '2016-07-01'), ('2016-07-01', '2016-08-01'),
          ('2016-08-01', '2016-09-01'), ('2016-09-01', '2016-10-01'), ('2016-10-01', '2016-11-01')]
experiments = [(65,1), (baseContextLength+24,24), (baseContextLength+48,48), (baseContextLength+72,72)]

In [60]:
dfCompl = pd.read_csv(f"{path}/train.csv", parse_dates=["date"], index_col=None)
for exp in experiments:
    maes = []
    rmses = []
    mapes = []

    ###define prediction length, i.e., number of forecast steps
    ###define prediction start step, it's N to last step in your data
    pred_len = pred_start =  exp[1]
    data_len = exp[0]
    #if i >= 2:
    #    data_len = 3 * pred_len

    for split in splits:
        dfSplit = dfCompl.iloc[(dfCompl["date"] > pd.to_datetime(split[0]).tz_localize(pytz.timezone('US/Eastern'))).values].reset_index(drop=True)
        dfSplit = dfCompl.iloc[(dfCompl["date"] < pd.to_datetime(split[1]).tz_localize(pytz.timezone('US/Eastern'))).values].reset_index(drop=True)
        
    
        df = dfSplit.iloc[range(len(dfSplit)-(data_len),len(dfSplit))]
        df = df.reset_index(drop=True)
    
        timefeat = df.columns[0]
        targets = [df.columns[1]]
        covariates = df.columns[range(2,len(df.columns))].array

        res = zeroshot_api.forecast(df, targets, covariates, timefeat, pred_len, pred_start, modelpath, autodiff=autodiff)
        if PLOT and res['ret'] == 'success':
            zeroshot_api.plot_res(res)

        trueData = np.array(res["targets"][0]["values"][(data_len-pred_len):])
        preds = np.array(res["targets"][0]["preds"])

        MAE = np.mean(np.abs(trueData - preds))
        RMSE = np.sqrt(np.mean((trueData - preds)**2))
        MAPE = np.mean(np.abs((trueData - preds) / trueData)) * 100

        maes.append(MAE)
        rmses.append(RMSE)
        mapes.append(MAPE)

    print(f"Average results with prediction length {pred_len} and context length {data_len - pred_len}")
    print(f"MAE: {np.mean(maes)}")
    print(f"RMSE: {np.mean(rmses)}")
    print(f"MAPE: {np.mean(mapes)}")

Average results with prediction length 1 and context length 64
MAE: 391.8100956231321
RMSE: 391.8100956231321
MAPE: 6.7061014409932715
Average results with prediction length 24 and context length 288
MAE: 870.7537413571099
RMSE: 1052.042918192833
MAPE: 15.05966120558287
Average results with prediction length 48 and context length 288
MAE: 1041.4379718717378
RMSE: 1245.0214995105148
MAPE: 16.712135451771896
Average results with prediction length 72 and context length 288
MAE: 1026.4130190761066
RMSE: 1269.1510634143535
MAPE: 17.5446979055435
